Import Libraries

In [1]:
import requests
import numpy as np
from bs4 import BeautifulSoup
import urllib.request
import math 
import cv2
import pytesseract
import re
from time import time
from decimal import ROUND_UP
from math import ceil
import pandas as pd
import sys, os

User Inputs

In [2]:
#application_path = os.path.dirname(sys.executable)

URL = input('URL: ')
# userMaxPrice = input('Enter Maximum Price: ')
# priceIterator = input('Price Iterator: ')
# minBedNo = input('Minimum number of bedrooms: ')

userMaxPrice = '650000'
priceIterator = '50000'
minBedNo = '2'

# splits the URL to the relevant area only and ignores all the features selected on the website
beforeURL = URL.split('%')[0] + '%'
afterURL = URL.split('%')[1]
URL = afterURL.split('&')[0]
generatedURL = beforeURL + URL + '&minBedrooms=' + minBedNo
print(generatedURL)

https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E1261&minBedrooms=2


Get all the property addresses

In [3]:
# rightmove main URL
### initialise search ### 

allLinks = np.array([])
allLinksCombined = np.array([])
cleanLinks = np.array([])
minPrice = 50000
maxPrice = minPrice + int(priceIterator)

pageIterator = (int(userMaxPrice)-minPrice)/int(priceIterator)
for x in range(int(pageIterator)):
    URL = generatedURL + '&maxPrice=' + str(maxPrice) + '&minPrice=' + str(minPrice)
    page = requests.get(URL)

    # parse search results card
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id='l-container')

    # index for iterator (remember each page has 24 ads)
    noOfResults = soup.find('div', attrs={'id': 'searchHeader'})
    noOfResults = noOfResults.select('span')[0].text
    noOfResults = re.sub(r',', '', noOfResults)
    noOfResults = ceil(int(noOfResults)/24) # round up to nearest page (note max page no is 42)

    print(noOfResults, " Pages available for min price = ", minPrice, " max price = ", maxPrice)

    i = 0
    all_ids = np.array([])
    for x in range(noOfResults):
        URL = generatedURL + '&maxPrice=' + str(maxPrice) + '&minPrice=' + str(minPrice) + '&sortType=18&index='+ str(i)
        print('Scrapped Page No.', x+1)
        page = requests.get(URL)

        # parse search results card
        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find(id='l-container')

        # Extract and store ids
        links = results.select('a')
        for card in links:
            ids = card.get('id')
            ids = ids.strip() if ids is not None else ''
            all_ids = np.append(all_ids, ids)

        # clean array and add links
        all_ids = all_ids[all_ids != '']
        cleanIds = np.char.strip(all_ids, 'prop')
        allLinks = np.char.add(cleanIds, '#/?channel=RES_BUY')
        allLinks = np.char.add('https://www.rightmove.co.uk/properties/', allLinks)
    
        i = i + 24

    allLinksCombined = np.append(allLinksCombined, allLinks)
    minPrice = minPrice + int(priceIterator) 
    maxPrice = maxPrice + int(priceIterator) 

# removes empty and repeated links 
cleanLinks = np.char.replace(allLinksCombined, 'https://www.rightmove.co.uk/properties/0#/?channel=RES_BUY', '')
cleanLinks = cleanLinks[cleanLinks != '']
cleanLinks = np.unique(cleanLinks)

print('Number of links acquired: ', len(cleanLinks))
print(cleanLinks)


0  Pages available for min price =  50000  max price =  100000
0  Pages available for min price =  100000  max price =  150000
0  Pages available for min price =  150000  max price =  200000
1  Pages available for min price =  200000  max price =  250000
Scrapped Page No. 1
1  Pages available for min price =  250000  max price =  300000
Scrapped Page No. 1
1  Pages available for min price =  300000  max price =  350000
Scrapped Page No. 1
1  Pages available for min price =  350000  max price =  400000
Scrapped Page No. 1
2  Pages available for min price =  400000  max price =  450000
Scrapped Page No. 1
Scrapped Page No. 2
2  Pages available for min price =  450000  max price =  500000
Scrapped Page No. 1
Scrapped Page No. 2
1  Pages available for min price =  500000  max price =  550000
Scrapped Page No. 1
2  Pages available for min price =  550000  max price =  600000
Scrapped Page No. 1
Scrapped Page No. 2
1  Pages available for min price =  600000  max price =  650000
Scrapped Page

Fishing Properties

In [14]:
i = 0
counter = 0
all_info = np.array([])
data = np.array([])

for property in cleanLinks:
    start = time()
    ad = requests.get(property)
    soup = BeautifulSoup(ad.content, "html.parser")

    # ensures only freehold properties
    if soup.select('p')[4].text.lower() == 'freehold' or soup.select('p')[6].text.lower() == 'freehold':

        # extract property data
        propertyAddress = soup.select('h1')[0].text
        propertyType = soup.select('p')[0].text
        typeOfProperty = soup.select('p')[1].text
        noOfBeds = soup.select('p')[2].text
        priceLink = soup.body.find('div', attrs={'class': '_1gfnqJ3Vtd1z40MlC0MzXu'})
        price = priceLink.select('span')[0].text
        adDateLink = soup.body.find('div', attrs={'class': '_2nk2x6QhNB1UrxdI5KpvaF'})
        date = re.sub('<div class="_2nk2x6QhNB1UrxdI5KpvaF">', '', str(adDateLink))
        date = re.sub('</div>', '', str(date))
        newIds = re.findall(r'\b\d+\b', str(cleanLinks[i])) # use new property id
        mapLocation = np.char.add('https://www.rightmove.co.uk/properties/', str(newIds[0]))
        floorPlan = np.char.add(mapLocation, '#/floorplan?activePlan=1&channel=RES_BUY')
        mapLocation = np.char.add(mapLocation, '#/map?channel=RES_BUY')
        
        # filter through the images to find the floor plan link
        floorPlanImage = requests.get(floorPlan)
        floorPlanImage = BeautifulSoup(floorPlanImage.content, "html.parser")
        floorPlanImage = floorPlanImage.find('div', attrs={'id': 'root'})
        for j in range(len(floorPlanImage.select('img'))):
            nextFloorPlanImage = floorPlanImage.select('img')[j].get_attribute_list('src',str)
            nextFloorPlanImage = ' '.join(nextFloorPlanImage)
            if '_max_296x197' in nextFloorPlanImage:
                foundFloorPlanImage = nextFloorPlanImage.replace('_max_296x197', '')
                break
            else:
                foundFloorPlanImage = 'None'
                pass
        
        i = i+1
        end = time()
        print(str(i)+' out of '+str(len(cleanLinks))+f' It took {round(end - start,2)} seconds!') 

        data = [str(price), str(propertyAddress), str(typeOfProperty), str(noOfBeds), str(foundFloorPlanImage), str(mapLocation), str(date)]
        all_info = np.append(all_info, data)
        counter = counter + 1
    else:
        #print('skipped...')
        i = i+1
        end = time()
        print(str(i)+' out of '+str(len(cleanLinks))+f' It took {round(end - start,2)} seconds!') 
        continue

# split the array into equally divided arrays
all_info = np.split(all_info, counter)
print(all_info)

1 out of 114 It took 0.43 seconds!
2 out of 114 It took 0.43 seconds!
3 out of 114 It took 0.65 seconds!
4 out of 114 It took 0.86 seconds!
5 out of 114 It took 0.39 seconds!
6 out of 114 It took 0.52 seconds!
7 out of 114 It took 0.4 seconds!
8 out of 114 It took 1.61 seconds!
9 out of 114 It took 0.36 seconds!
10 out of 114 It took 0.37 seconds!
11 out of 114 It took 0.54 seconds!
12 out of 114 It took 0.72 seconds!
13 out of 114 It took 0.4 seconds!
14 out of 114 It took 1.5 seconds!
15 out of 114 It took 0.38 seconds!
16 out of 114 It took 0.4 seconds!
17 out of 114 It took 0.84 seconds!
18 out of 114 It took 0.53 seconds!
19 out of 114 It took 0.77 seconds!
20 out of 114 It took 1.46 seconds!
21 out of 114 It took 0.35 seconds!
22 out of 114 It took 0.75 seconds!
23 out of 114 It took 0.42 seconds!
24 out of 114 It took 1.49 seconds!
25 out of 114 It took 1.67 seconds!
26 out of 114 It took 0.82 seconds!
27 out of 114 It took 0.41 seconds!
28 out of 114 It took 0.38 seconds!
29 ou

Image Processing

In [22]:
import importlib 
from numpy import empty
import searchFunction
importlib.reload(searchFunction) # forces the definition to reload in case you make changes to it
from searchFunction import search 

updated_info = np.array([])

for info in range(len(all_info)):
    print(str(info+1)+' out of '+str(len(all_info)))
    try:
        if not all_info[info][4] == 'None':
            # Load the img
            req = urllib.request.urlopen(all_info[info][4])
            arr = np.asarray(bytearray(req.read()))
            img = cv2.imdecode(arr, -1) # 'Load it as it is'

            # Display loaded image
            # cv2.imshow('Floor Plan', img)
            # if cv2.waitKey() & 0xff == 27: quit()

            # OCR
            pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
            configs = r'--psm 6'
            try:
                rawText = pytesseract.image_to_string(img, config=configs)
            except:
                continue
            rawText = rawText.lower()
            rawText = re.sub(r',', '', str(rawText)) # removes all commas 
            rawText = re.sub(r'\b\.\d+\b', '', str(rawText)) # removes all ints
            #print(rawText)

            newText = search(rawText)
            #print(newText)
            newInfo = list(np.append(all_info[info], newText)) # adding list adds the ',' back into the string array    
            updated_info = np.append(updated_info, newInfo)
    except:
        continue        

# split the array into equally divided arrays and convert to pandas for further processing
updated_info = np.split(updated_info, len(updated_info)/8) 
updated_info = pd.DataFrame(updated_info, columns = ['price', 'propertyAddress', 'typeOfProperty', 'noOfBeds', 'foundFloorPlanImage', 'mapLocation', 'date', 'area'])
updated_info = updated_info.drop(columns='foundFloorPlanImage')
print(updated_info)

1 out of 25
2 out of 25
3 out of 25
4 out of 25
5 out of 25
6 out of 25
7 out of 25
8 out of 25
9 out of 25
10 out of 25
11 out of 25
12 out of 25
13 out of 25
14 out of 25
15 out of 25
16 out of 25
17 out of 25
18 out of 25
19 out of 25
20 out of 25
21 out of 25
22 out of 25
23 out of 25
24 out of 25
25 out of 25
       price                              propertyAddress  \
0   £575,000                 Curzon Avenue, Stanmore, HA7   
1   £650,000                          Stanmore, Middlesex   
2   £595,000                Daventer Drive, Stanmore, HA7   
3   £585,000      Bellamy Drive, Stanmore, Middlesex, HA7   
4   £650,000             St. Edmunds Drive, Stanmore, HA7   
5   £585,000                     Stanmore, Middlesex, HA7   
6   £650,000                   Eaton Close, Stanmore, HA7   
7   £600,000               Brightwen Grove, Stanmore, HA7   
8   £585,000                 Ashdale Grove, Stanmore, HA7   
9   £565,000              Castellane Close, Stanmore, HA7   
10  £530,000 

In [8]:
# this should load the old file and append any new data found to it making sure it's all unique data
# and not repeated 

updated_info.to_csv('output.csv', index=False) # comment out if you want to use the old data

loadData = pd.read_csv(r'output.csv')
frames = [loadData, updated_info]
combinedData = pd.concat(frames)
combinedData = combinedData.drop_duplicates(subset=['mapLocation'])

combinedData['price'] = combinedData['price'].replace('[\£,]', '', regex=True).astype(float)
combinedData['area'] = combinedData['area'].astype(float)
combinedData['priceRank'] = combinedData['price'].rank(ascending=1)
combinedData['areaRank'] = combinedData['area'].rank(ascending=0)
combinedData['Rank Sum'] = combinedData.priceRank + combinedData.areaRank
combinedData = combinedData.sort_values(by = 'Rank Sum', ascending = True)

combinedData.to_csv('output_processed.csv', index=False)
print(combinedData)

       price                                 propertyAddress  typeOfProperty  \
2   750000.0                         Manger Road, London, N7           House   
3   525000.0  Laffitte House, New Orleans Walk, Hornsey Rise       Apartment   
12  630000.0                       Parkside Crescent, London        Terraced   
4   750000.0                             Blackstock Road, N4        Terraced   
15  750000.0                         Russet Crescent, London  End of Terrace   
11  750000.0                     Russet Crescent, London, N7  End of Terrace   
5   750000.0          Russet Crescent, Islington, London, N7           House   
9   500000.0                         Tollington Park, London       Apartment   
6   600000.0                    Fonthill Road, Finsbury Park        Terraced   
10  800000.0                   Pavillion Mews, Finsbury Park        Terraced   
13  475000.0                         Burder Road, London, N1            Flat   
8   675000.0                  Mildmay Gr